In [1]:
import pandas as pd
import requests

In [2]:
df = pd.DataFrame()

In [3]:
for i in range(1,471):
    resp1 = requests.get('https://api.themoviedb.org/3/movie/top_rated?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US&page={}'.format(i))
    tmp_df1 = pd.DataFrame(resp1.json()['results'])[['title','overview','genre_ids']]
    df = df.append(tmp_df1,ignore_index=True)

C:\Users\Insha sadaf\AppData\Local\Temp\ipykernel_11880\1010239562.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp_df1,ignore_index=True)
C:\Users\Insha sadaf\AppData\Local\Temp\ipykernel_11880\1010239562.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp_df1,ignore_index=True)
C:\Users\Insha sadaf\AppData\Local\Temp\ipykernel_11880\1010239562.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp_df1,ignore_index=True)
C:\Users\Insha sadaf\AppData\Local\Temp\ipykernel_11880\1010239562.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp_df1,ignore_index=True)


In [4]:
## this is used for rename columns name
# df.rename(columns = {'genre_ids':'id'}, inplace = True)abs

In [11]:
df_exploded = df.explode('genre_ids')

In [35]:
df_exploded
# df

,title,overview,genre_ids
0,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,18
0,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,80
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",18
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",80
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,18
...,...,...,...
9397,Dragonball Evolution,"On his 18th birthday, Goku receives a mystical...",878
9397,Dragonball Evolution,"On his 18th birthday, Goku receives a mystical...",53
9398,Birdemic: Shock and Terror,A platoon of eagles and vultures attacks the r...,10749
9398,Birdemic: Shock and Terror,A platoon of eagles and vultures attacks the r...,27


In [41]:
resp2 = requests.get('https://api.themoviedb.org/3/genre/movie/list?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US')
df2 = pd.DataFrame(resp2.json()['genres'])

In [43]:
df2

,id,name
0,28,Action
1,12,Adventure
2,16,Animation
3,35,Comedy
4,80,Crime
5,99,Documentary
6,18,Drama
7,10751,Family
8,14,Fantasy
9,36,History


In [44]:
# Merge the DataFrames based on the 'genre_id' column
merged_df = pd.merge(df_exploded,df2,left_on='genre_ids', right_on='id',how='left')

# Select the desired columns
final_df = merged_df[['title', 'overview', 'name']]
final_df

final_df['name'] = final_df['name'].fillna('')

# Convert all values in 'name' column to string
final_df['name'] = final_df['name'].astype(str)

C:\Users\Insha sadaf\AppData\Local\Temp\ipykernel_11880\2679544845.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['name'] = final_df['name'].fillna('')
C:\Users\Insha sadaf\AppData\Local\Temp\ipykernel_11880\2679544845.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['name'] = final_df['name'].astype(str)


In [45]:
df_final = final_df.groupby(['title','overview'])['name'].apply(lambda x: ', '.join(x)).reset_index()
df_final

,title,overview,name
0,#Alive,"As a grisly virus rampages a city, a lone man ...","Horror, Action, Adventure, Thriller"
1,#realityhigh,When nerdy high schooler Dani finally attracts...,Comedy
2,'71,A young British soldier must find his way back...,"Thriller, Action, Drama, War"
3,(500) Days of Summer,"Tom, greeting-card writer and hopeless romanti...","Comedy, Drama, Romance"
4,(T)Raumschiff Surprise - Periode 1,Hundreds of years after humans have settled on...,"Comedy, Science Fiction"
...,...,...,...
9389,xXx,Xander Cage is your standard adrenaline junkie...,"Action, Adventure, Thriller, Crime"
9390,xXx: Return of Xander Cage,Extreme athlete turned government operative Xa...,"Action, Adventure, Crime"
9391,xXx: State of the Union,Darius Stone's criminal record and extreme spo...,"Action, Adventure, Crime, Mystery, Thriller"
9392,¡Three Amigos!,A trio of unemployed silent film actors are mi...,"Comedy, Western, Drama"


# Text Preprocessing on overview:

 **Lowercasing**

In [46]:
df_final['overview'] = df['overview'].str.lower()
df_final

,title,overview,name
0,#Alive,imprisoned in the 1940s for the double murder ...,"Horror, Action, Adventure, Thriller"
1,#realityhigh,"spanning the years 1945 to 1955, a chronicle o...",Comedy
2,'71,in the continuing saga of the corleone crime f...,"Thriller, Action, Drama, War"
3,(500) Days of Summer,the true story of how businessman oskar schind...,"Comedy, Drama, Romance"
4,(T)Raumschiff Surprise - Periode 1,the defense and the prosecution have rested an...,"Comedy, Science Fiction"
...,...,...,...
9389,xXx,when four bodies are discovered among the indu...,"Action, Adventure, Thriller, Crime"
9390,xXx: Return of Xander Cage,a teenager and his mother find themselves besi...,"Action, Adventure, Crime"
9391,xXx: State of the Union,joe and averell are the eldest and youngest of...,"Action, Adventure, Crime, Mystery, Thriller"
9392,¡Three Amigos!,luke skywalker and han solo battle evil imperi...,"Comedy, Western, Drama"


**Removing HTML-tags**

In [47]:
import re
def remove_html_tags(text):
    pattern = re.compile('<.*>')
    return pattern.sub(r'', text)

In [48]:
df_final['overview'] = df_final['overview'].apply(remove_html_tags)
df_final

,title,overview,name
0,#Alive,imprisoned in the 1940s for the double murder ...,"Horror, Action, Adventure, Thriller"
1,#realityhigh,"spanning the years 1945 to 1955, a chronicle o...",Comedy
2,'71,in the continuing saga of the corleone crime f...,"Thriller, Action, Drama, War"
3,(500) Days of Summer,the true story of how businessman oskar schind...,"Comedy, Drama, Romance"
4,(T)Raumschiff Surprise - Periode 1,the defense and the prosecution have rested an...,"Comedy, Science Fiction"
...,...,...,...
9389,xXx,when four bodies are discovered among the indu...,"Action, Adventure, Thriller, Crime"
9390,xXx: Return of Xander Cage,a teenager and his mother find themselves besi...,"Action, Adventure, Crime"
9391,xXx: State of the Union,joe and averell are the eldest and youngest of...,"Action, Adventure, Crime, Mystery, Thriller"
9392,¡Three Amigos!,luke skywalker and han solo battle evil imperi...,"Comedy, Western, Drama"


**Remove URLs**

In [49]:
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

In [50]:
df_final['overview'] = df_final['overview'].apply(remove_url)
df_final

,title,overview,name
0,#Alive,imprisoned in the 1940s for the double murder ...,"Horror, Action, Adventure, Thriller"
1,#realityhigh,"spanning the years 1945 to 1955, a chronicle o...",Comedy
2,'71,in the continuing saga of the corleone crime f...,"Thriller, Action, Drama, War"
3,(500) Days of Summer,the true story of how businessman oskar schind...,"Comedy, Drama, Romance"
4,(T)Raumschiff Surprise - Periode 1,the defense and the prosecution have rested an...,"Comedy, Science Fiction"
...,...,...,...
9389,xXx,when four bodies are discovered among the indu...,"Action, Adventure, Thriller, Crime"
9390,xXx: Return of Xander Cage,a teenager and his mother find themselves besi...,"Action, Adventure, Crime"
9391,xXx: State of the Union,joe and averell are the eldest and youngest of...,"Action, Adventure, Crime, Mystery, Thriller"
9392,¡Three Amigos!,luke skywalker and han solo battle evil imperi...,"Comedy, Western, Drama"


**Remove Punctuation**

In [51]:
import string,time
exclude = string.punctuation

In [52]:
def remove_punc1(text):
    return text.translate(str.maketrans('','',exclude))

In [53]:
df_final['overview'] = df_final['overview'].apply(remove_punc1)
df_final

,title,overview,name
0,#Alive,imprisoned in the 1940s for the double murder ...,"Horror, Action, Adventure, Thriller"
1,#realityhigh,spanning the years 1945 to 1955 a chronicle of...,Comedy
2,'71,in the continuing saga of the corleone crime f...,"Thriller, Action, Drama, War"
3,(500) Days of Summer,the true story of how businessman oskar schind...,"Comedy, Drama, Romance"
4,(T)Raumschiff Surprise - Periode 1,the defense and the prosecution have rested an...,"Comedy, Science Fiction"
...,...,...,...
9389,xXx,when four bodies are discovered among the indu...,"Action, Adventure, Thriller, Crime"
9390,xXx: Return of Xander Cage,a teenager and his mother find themselves besi...,"Action, Adventure, Crime"
9391,xXx: State of the Union,joe and averell are the eldest and youngest of...,"Action, Adventure, Crime, Mystery, Thriller"
9392,¡Three Amigos!,luke skywalker and han solo battle evil imperi...,"Comedy, Western, Drama"


**Spell Correction**

In [55]:
# from textblob import TextBlob

**Stopper removal**

In [66]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Insha
[nltk_data]     sadaf\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [67]:
from nltk.corpus import stopwords

In [68]:
def remove_stopwords(text):
    new_text = []
    
    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)

In [69]:
df_final['overview'] = df_final['overview'].apply(remove_stopwords)
df_final

,title,overview,name
0,#Alive,imprisoned 1940s double murder wife lo...,"Horror, Action, Adventure, Thriller"
1,#realityhigh,spanning years 1945 1955 chronicle fictio...,Comedy
2,'71,continuing saga corleone crime family you...,"Thriller, Action, Drama, War"
3,(500) Days of Summer,true story businessman oskar schindler save...,"Comedy, Drama, Romance"
4,(T)Raumschiff Surprise - Periode 1,defense prosecution rested jury filing ...,"Comedy, Science Fiction"
...,...,...,...
9389,xXx,four bodies discovered among industrial dec...,"Action, Adventure, Thriller, Crime"
9390,xXx: Return of Xander Cage,teenager mother find besieged threatening...,"Action, Adventure, Crime"
9391,xXx: State of the Union,joe averell eldest youngest four dalton ...,"Action, Adventure, Crime, Mystery, Thriller"
9392,¡Three Amigos!,luke skywalker han solo battle evil imperial ...,"Comedy, Western, Drama"


**Tokenization**

In [70]:
from nltk.tokenize import word_tokenize, sent_tokenize

In [71]:
word_tokenize(df_final['overview'] )

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\Insha sadaf/nltk_data'
    - 'C:\\Users\\Insha sadaf\\anaconda3\\nltk_data'
    - 'C:\\Users\\Insha sadaf\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\Insha sadaf\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Insha sadaf\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************
